<a href="https://colab.research.google.com/github/javohir-ergashboyev/tensorflow_basics/blob/main/Hypertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this particular process, simple image classification model are built using hyperparametres library

1 setup and installing

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -U -q keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.5 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

In [4]:
from tensorflow.keras.datasets import fashion_mnist

In [5]:
data=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
type(data[0][1])

numpy.ndarray

In [7]:
(xtrain,ytrain),(xtest,ytest)=data
xtrain.shape, xtest.shape

((60000, 28, 28), (10000, 28, 28))

In [19]:
xtrain=xtrain.astype('float64')/255.0
xtest=xtest.astype('float64')/255.0

In [20]:
def model_builder(hp):
  model=keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  hp_units=hp.Int('units',min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))
  hp_rate=hp.Choice('learning_rate', values=[1e-2,1e-3,1e-4])
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=hp_rate),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy']
  )
  return model

In [21]:
tuner=kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='hypertuning'
)

In [22]:
stop_early=keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [23]:
hp_search=tuner.search(xtrain,ytrain, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_parmetres=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"best unit is {best_parmetres.get('units')}, and best learning rate is {best_parmetres.get('learning_rate')} ")

Trial 30 Complete [00h 00m 47s]
val_accuracy: 0.7714166641235352

Best val_accuracy So Far: 0.8826666474342346
Total elapsed time: 00h 12m 14s
best unit is 320, and best learning rate is 0.01 


In [25]:
model=tuner.hypermodel.build(best_parmetres)
history=model.fit(xtrain,ytrain,epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.6130 - accuracy: 0.7785 - val_loss: 0.4753 - val_accuracy: 0.8285
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4310 - accuracy: 0.8444 - val_loss: 0.4065 - val_accuracy: 0.8501
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3913 - accuracy: 0.8556 - val_loss: 0.3963 - val_accuracy: 0.8557
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3647 - accuracy: 0.8661 - val_loss: 0.3798 - val_accuracy: 0.8643
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3439 - accuracy: 0.8741 - val_loss: 0.3642 - val_accuracy: 0.8663
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3273 - accuracy: 0.8787 - val_loss: 0.3558 - val_accuracy: 0.8723
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3157 - accuracy: 0.8835 - val_loss: 0.3573 - val_accuracy:

In [26]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 40
